In [ ]:
##my final code
import cv2
import numpy as np
import time
from ultralytics import YOLO
from shapely.geometry import Polygon
from base64 import b64encode
from IPython.display import HTML

# --- Global variables for stabilized line positions ---
stable_road_bottom_y = None
stable_sky_bottom_y = None

def find_road_extent(frame):
    global stable_road_bottom_y, stable_sky_bottom_y
    
    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color ranges for road and sky
    road_color_range = (np.array([0, 0, 0]), np.array([179, 255, 100]))  # Adjust as needed
    sky_color_range = (np.array([90, 0, 0]), np.array([120, 255, 255]))  # Adjust as needed
    
    # Mask for road and sky regions
    road_mask = cv2.inRange(hsv, road_color_range[0], road_color_range[1])
    sky_mask = cv2.inRange(hsv, sky_color_range[0], sky_color_range[1])
    
    # Find contours in the masks
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    road_contours = max(contours, key=cv2.contourArea) if contours else None
    
    contours, _ = cv2.findContours(sky_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sky_contours = max(contours, key=cv2.contourArea) if contours else None
    
    # Find the lowest points in the contours
    road_bottom_y = find_lowest_y_coordinate(road_contours) if road_contours is not None else float('inf')
    sky_bottom_y = find_lowest_y_coordinate(sky_contours) if sky_contours is not None else float('inf')
    
    # Midway line position
    midway_y = frame.shape[0] // 2
    
    # Check if the detected line is in the middle one-third of the screen
    middle_third_y = frame.shape[0] * 2 / 3
    
    if middle_third_y / 3 < road_bottom_y < middle_third_y:
        stable_road_bottom_y = road_bottom_y
    
    if middle_third_y / 3 < sky_bottom_y < middle_third_y:
        stable_sky_bottom_y = sky_bottom_y
    
    return stable_road_bottom_y, stable_sky_bottom_y, midway_y

def find_lowest_y_coordinate(contour):
    lowest_y = float('inf')
    if contour is not None:
        for point in contour[:, 0]:
            y = point[1]
            if y < lowest_y:
                lowest_y = y
    return lowest_y if lowest_y != float('inf') else None

def calculate_polygon_area(points):
    """Calculate the area of a polygon given its vertices."""
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def get_intersection_area(box, trapezium_pts):
    """Calculate the intersection area between a bounding box and a trapezium using Shapely."""
    box_polygon = Polygon([
        (box[0], box[1]),
        (box[2], box[1]),
        (box[2], box[3]),
        (box[0], box[3])
    ])
    trapezium_polygon = Polygon(trapezium_pts)

    intersection = box_polygon.intersection(trapezium_polygon)
    return intersection.area if not intersection.is_empty else 0

# --- Load YOLOv8 Model ---
model_path = 'my_model2.pt'  
model = YOLO(model_path)

# --- Load Video ---
video_path = 'crash!.mp4'  
cap = cv2.VideoCapture(video_path)

# --- Video Output Setup ---
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_video_path = 'output_video.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# --- TTC Calculation Parameters ---
avg_vehicle_length = 4.5
camera_fov = 60  
ttc_threshold = 0.6  
time_interval = 0.1  
font = cv2.FONT_HERSHEY_SIMPLEX

# --- Warning Parameters ---
ego_velocity = 25  

# --- Tracking Data ---
prev_positions = {}

# --- Main Processing Loop ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # --- YOLOv8 Object Detection ---
    results = model(frame)

    # --- Find road extent in the frame based on color segmentation ---
    road_bottom_y, sky_bottom_y, midway_y = find_road_extent(frame)

    # --- Calculate the width of the base ---
    base_width = frame.shape[1] // 3
    
    # Determine the lower extent considering road, sky, and midway lines
    lower_y = None
    if stable_road_bottom_y is not None and stable_sky_bottom_y is not None:
        lower_y = max(stable_road_bottom_y, stable_sky_bottom_y, midway_y)
    elif stable_road_bottom_y is not None:
        lower_y = max(stable_road_bottom_y, midway_y)
    elif stable_sky_bottom_y is not None:
        lower_y = max(stable_sky_bottom_y, midway_y)
    else:
        lower_y = midway_y
    
    # Calculate top edge of the trapezium
    top_y = frame.shape[0]
    if lower_y is not None:
        top_y = int(lower_y)
    
    # Draw a trapezium if lower_y is valid
    trapezium_pts = None
    if lower_y is not None:
        top_width = base_width * 0.4
        top_width_half = top_width / 2
        trapezium_pts = np.array([[frame.shape[1] / 2 - base_width / 2, frame.shape[0]],
                                  [frame.shape[1] / 2 + base_width / 2, frame.shape[0]],
                                  [frame.shape[1] / 2 + top_width_half, top_y],
                                  [frame.shape[1] / 2 - top_width_half, top_y]], np.int32)
        trapezium_pts = trapezium_pts.reshape((-1, 1, 2))
        overlay = frame.copy()
        cv2.fillPoly(overlay, [trapezium_pts], (210, 128, 255))
        alpha = 0.4
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    
    # Visualize the extent on the frame
    if stable_road_bottom_y is not None:
        cv2.line(frame, (0, int(stable_road_bottom_y)), (frame.shape[1], int(stable_road_bottom_y)), (0, 255, 0), 1)
    if stable_sky_bottom_y is not None:
        cv2.line(frame, (0, int(stable_sky_bottom_y)), (frame.shape[1], int(stable_sky_bottom_y)), (255, 0, 0), 1)
    cv2.line(frame, (0, midway_y), (frame.shape[1], midway_y), (255, 255, 255), 1)
    
    # --- Calculate the area of the trapezium ---
    trapezium_area = calculate_polygon_area(trapezium_pts[:, 0]) if trapezium_pts is not None else 0
    
    # --- TTC, Speed Calculation & Display ---
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            score = box.conf[0].item()
            cls = box.cls[0].item()
            center_x = (x1 + x2) / 2

            # --- TTC Calculation ---
            obj_width = x2 - x1
            distance = (avg_vehicle_length * frame.shape[1]) / (2 * obj_width * np.tan(np.radians(camera_fov / 2)))

            if center_x in prev_positions:
                prev_time, prev_pos, prev_v = prev_positions[center_x]
                if time.time() - prev_time >= time_interval:
                    pixel_distance = np.sqrt((center_x - prev_pos[0])**2 + ((y1 + y2) // 2 - prev_pos[1])**2)
                    velocity_ms = (pixel_distance / (time.time() - prev_time)) * 0.02645833

                    # Acceleration Calculation
                    acceleration = (velocity_ms - prev_v) / time_interval

                    # TTC Calculation with Acceleration
                    relative_velocity = velocity_ms
                    if relative_velocity != 0:
                        ttc = (-relative_velocity + np.sqrt(relative_velocity**2 + 2 * acceleration * distance)) / acceleration
                        ttc = max(0, ttc)
                    else:
                        ttc = 999
            else:
                velocity_ms = 20  # Replace with actual velocity estimation
                ttc = distance / velocity_ms if velocity_ms > 0 else 999

            # Store current position and velocity
            prev_positions[center_x] = (time.time(), (center_x, (y1 + y2) // 2), velocity_ms)

            # --- Check for cut-in warning ---
            cut_in = False
            if trapezium_pts is not None:
                intersection_area = get_intersection_area([x1, y1, x2, y2], trapezium_pts[:, 0])
                
                if trapezium_area > 0 and intersection_area / trapezium_area > 0.15:  # 20% threshold
                    cut_in = True
                    cut_in_info = "Vehicle has cut in"
                    cut_in_color = (128, 0, 128)  # Dark purple
                    cv2.putText(frame, cut_in_info, (int(x1), int(y1) - 30), font, 1, cut_in_color, 2)  # Larger font size and thickness

            # --- Display TTC if less than threshold and vehicle has cut in ---
            if cut_in and ttc < ttc_threshold: 
                ttc_info = f"WARNING: {ttc:.2f}s"
                ttc_color = (0, 0, 255)
                cv2.putText(frame, ttc_info, (int(x1), int(y1) - 10), font, 1, ttc_color, 2)  # Larger font size and thickness
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Red bounding box for warning
            else:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Default green bounding box

            # Draw label
            label = model.names[int(cls)]
            cv2.putText(frame, label, (int(x1), int(y1) - 5), font, 0.5, (255, 255, 255), 1)

    # --- Display and Write Processed Frame ---
    out.write(frame)
    cv2.imshow('Processed Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Release Video and Display ---
cap.release()
out.release()
cv2.destroyAllWindows()

# --- Provide Download Link ---
def create_download_link(filename):
    with open(filename, "rb") as f:
        data = f.read()
    b64_data = b64encode(data).decode()
    return f'<a download="{filename}" href="data:video/mp4;base64,{b64_data}">Download Output Video</a>'

HTML(create_download_link(output_video_path))
